In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import sqlite3
import pandas as pd

# Connect to the database
conn = sqlite3.connect('/content/drive/MyDrive/5. Week 5 - SQL & Web Scrapping/sql-murder-mystery.db')  # Change this to your actual path
cursor = conn.cursor()

# Function to run SQL queries and display results
def run_query(query):
    return pd.read_sql_query(query, conn)

In [43]:
# List all tables in the database
tables_query = "SELECT name FROM sqlite_master WHERE type='table';"
tables = run_query(tables_query)
print("Tables in the database:")
print(tables)

Tables in the database:
                     name
0      crime_scene_report
1         drivers_license
2                  person
3  facebook_event_checkin
4               interview
5      get_fit_now_member
6    get_fit_now_check_in
7                  income
8                solution


In [20]:
# Retrieve the crime scene report
crime_scene_query = """
SELECT *
FROM crime_scene_report
WHERE city = "SQL City" and type = "murder"
ORDER BY date;
"""
crime_scene_report = run_query(crime_scene_query)
print("Crime Scene Report:")
print(crime_scene_report)

Crime Scene Report:
       date    type  \
0  20180115  murder   
1  20180215  murder   
2  20180215  murder   

                                                                                                                                                                                 description  \
0  Security footage shows that there were 2 witnesses. The first witness lives at the last house on "Northwestern Dr". The second witness, named Annabel, lives somewhere on "Franklin Ave".   
1                                                                                                                                                                 REDACTED REDACTED REDACTED   
2                                                                                                                                    Someone killed the guard! He took an arrow to the knee!   

       city  
0  SQL City  
1  SQL City  
2  SQL City  


In [19]:
# Get witness personal details
# One witness lives at the last house on "Northwestern Dr"
witness1_query = """
SELECT *
FROM person
WHERE address_street_name = 'Northwestern Dr'
ORDER BY address_number DESC
LIMIT 1
"""
witness1 = run_query(witness1_query)
print("\nWitness 1 Personal Details:")
print(witness1)

# The second witness, named Annabel, lives somewhere on "Franklin Ave"
witness2_query = """
SELECT *
FROM person
WHERE name LIKE 'Annabel%' AND address_street_name = 'Franklin Ave'
"""
witness2 = run_query(witness2_query)
print("\nWitness 2 Personal Details:")
print(witness2)


Witness 1 Personal Details:
      id            name  license_id  address_number address_street_name  \
0  14887  Morty Schapiro      118009            4919     Northwestern Dr   

         ssn  
0  111564949  

Witness 2 Personal Details:
      id            name  license_id  address_number address_street_name  \
0  16371  Annabel Miller      490173             103        Franklin Ave   

         ssn  
0  318771143  


In [23]:
witness_ids = witness1['id'].tolist() + witness2['id'].tolist()
print(witness_ids)

[14887, 16371]


In [26]:
# View witness interviews
witness_interviews_query = f"""
SELECT *
FROM interview
WHERE person_id IN ({','.join(map(str, witness_ids))})
"""
witness_interviews = run_query(witness_interviews_query)
print("\nWitness Interviews:")
print(witness_interviews)


Witness Interviews:
   person_id  \
0      14887   
1      16371   

                                                                                                                                                                                                                        transcript  
0  I heard a gunshot and then saw a man run out. He had a "Get Fit Now Gym" bag. The membership number on the bag started with "48Z". Only gold members have those bags. The man got into a car with a plate that included "H42W".  
1                                                                                                            I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th.  


In [27]:
# Check the gym database
# Get gym memberships of the identified witnesses
gym_database_query = f"""
SELECT *
FROM get_fit_now_member
WHERE person_id IN ({','.join(map(str, witness_ids))})
"""
gym_database = run_query(gym_database_query)
print("\nGym Database:")
print(gym_database)


Gym Database:
      id  person_id            name  membership_start_date membership_status
0  90081      16371  Annabel Miller               20160208              gold


In [29]:
# check gold membership and check in time
check_in_query = """SELECT *
FROM get_fit_now_check_in
WHERE membership_id like '%48Z%' AND check_in_date = 20180109
ORDER By check_in_date;"""

check_in_reports = run_query(check_in_query)
print("\nCheck in Times:")
print(check_in_reports)


Check in Times:
  membership_id  check_in_date  check_in_time  check_out_time
0         48Z7A       20180109           1600            1730
1         48Z55       20180109           1530            1700


In [30]:
# Check car details
# Get car details of the identified witnesses
car_details_query = f"""
SELECT *
FROM drivers_license
WHERE plate_number like "%H42W%";
"""
car_details = run_query(car_details_query)
print("\nCar Details:")
print(car_details)


Car Details:
       id  age  height eye_color hair_color  gender plate_number   car_make  \
0  183779   21      65      blue     blonde  female       H42W0X     Toyota   
1  423327   30      70     brown      brown    male       0H42W2  Chevrolet   
2  664760   21      71     black      black    male       4H42WR     Nissan   

  car_model  
0     Prius  
1  Spark LS  
2    Altima  


In [32]:
# The killer is a male so selecting id = 423327 and 664760
details_by_witness_query = """
SELECT *
FROM Person
WHERE license_id ="423327" OR license_id ="664760";
"""

witness_reported_details = run_query(details_by_witness_query)
print("\n Details Mentioned:")
print(witness_reported_details)


 Details Mentioned:
      id            name  license_id  address_number    address_street_name  \
0  51739  Tushar Chandra      664760             312                 Phi St   
1  67318   Jeremy Bowers      423327             530  Washington Pl, Apt 3A   

         ssn  
0  137882671  
1  871539279  


In [34]:
# checking gold membership for the above mentioned persons
check_gold_membership = """
SELECT *
FROM get_fit_now_member
WHERE person_id ="51739" OR person_id ="67318";
"""

gold_membership_details = run_query(check_gold_membership)
print("\n Members with gold:")
print(gold_membership_details)


 Members with gold:
      id  person_id           name  membership_start_date membership_status
0  48Z55      67318  Jeremy Bowers               20160101              gold


In [37]:
#checking for testimony given by the above person
murder_testimony = """
SELECT *
FROM interview
WHERE person_id = 67318;
"""

murderer_testimony = run_query(murder_testimony)
print("\nTestimony:")
print(murderer_testimony)


Testimony:
   person_id  \
0      67318   

                                                                                                                                                                                                                                           transcript  
0  I was hired by a woman with a lot of money. I don't know her name but I know she's around 5'5" (65") or 5'7" (67"). She has red hair and she drives a Tesla Model S. I know that she attended the SQL Symphony Concert 3 times in December 2017.\n  


In [39]:
# checking for the persons driver licence who hired the murderer
murder_hire_licence_details = """
SELECT *
FROM drivers_license
WHERE car_make = "Tesla" AND car_model = "Model S" AND gender = "female"
AND hair_color = "red";
"""

details_hire = run_query(murder_hire_licence_details)
print("\nPossible hirees driver licences:")
print(details_hire)


Possible hirees:
       id  age  height eye_color hair_color  gender plate_number car_make  \
0  202298   68      66     green        red  female       500123    Tesla   
1  291182   65      66      blue        red  female       08CM64    Tesla   
2  918773   48      65     black        red  female       917UU3    Tesla   

  car_model  
0   Model S  
1   Model S  
2   Model S  


In [40]:
# checking for the hiree details
hiree_details = """
SELECT *
FROM Person
WHERE license_id ="202298" OR license_id ="291182" OR license_id ="918773";
"""

person_details = run_query(hiree_details)
print("\nPossible Hirees:")
print(person_details)


Possible Hirees:
      id              name  license_id  address_number address_street_name  \
0  78881          Red Korb      918773             107         Camerata Dr   
1  90700     Regina George      291182             332           Maple Ave   
2  99716  Miranda Priestly      202298            1883          Golden Ave   

         ssn  
0  961388910  
1  337169072  
2  987756388  


In [41]:
# Getting details of the person who went to concert 3 times
person_concert_details = """
SELECT person_id, count(*),event_name
FROM facebook_event_checkin
Group BY person_id
having count(*) = 3 AND event_name = "SQL Symphony Concert" AND date like '%201712%';
"""

concert_attended = run_query(person_concert_details)
print("Persons who attended concert 3 times: ")
print(concert_attended)

Persons who attended concert 3 times: 
   person_id  count(*)            event_name
0      24556         3  SQL Symphony Concert
1      99716         3  SQL Symphony Concert


In [42]:
print("From the above two queries the person id that is common is 99716")
print("So the killer was hired by person id : 99716, i.e; Miranda Priestly")

From the above two queries the person id that is common is 99716
So the killer was hired by person id : 99716, i.e; Miranda Priestly
